In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [6]:
dropout_value = 0.0
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.convblock1 = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=1, bias=False),nn.ReLU(),nn.BatchNorm2d(10), nn.Dropout(dropout_value)) 
        self.convblock2 = nn.Sequential(nn.Conv2d(in_channels=10, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),nn.ReLU(),nn.BatchNorm2d(16), nn.Dropout(dropout_value))
        self.convblock3 = nn.Sequential(nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), bias=False))
        self.pool1 = nn.MaxPool2d(2,2)
        
        
        self.convblock4 = nn.Sequential(nn.Conv2d(in_channels= 10, out_channels= 16, kernel_size=(3, 3), padding=1, bias=False), nn.ReLU(),nn.BatchNorm2d(16), nn.Dropout(dropout_value))
        self.convblock5 = nn.Sequential(nn.Conv2d(in_channels=16, out_channels= 20, kernel_size=(3, 3), padding=1, bias=False),nn.ReLU(),nn.BatchNorm2d(20), nn.Dropout(dropout_value))
        self.convblock6 = nn.Sequential(nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(1, 1), bias=False))
        self.pool2 = nn.MaxPool2d(2, 2)
        self.convblock7 = nn.Sequential(nn.Conv2d(in_channels=10, out_channels= 16, kernel_size=(3, 3), bias=False),nn.ReLU(),nn.BatchNorm2d(16), nn.Dropout(dropout_value))
        self.gap = nn.Sequential(nn.AvgPool2d(kernel_size=5))
        self.convblock8 = nn.Sequential(nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), bias=False))
        
         

    def forward(self, x):
        x = self.convblock1(x) #input - 28; Output - 28; RF - 3; output channels - 10
        x = self.convblock2(x) #input - 28; Output - 28; RF - 5; output channels - 15
        x = self.convblock3(x)
        x = self.pool1(x) #input - 28; Output - 14; RF - 10; output channels - 15
        x = self.convblock4(x) #input - 14; Output - 14; RF - 12; output channels - 20
        x = self.convblock5(x)#input - 14; Output - 14; RF - 14; output channels - 25
        x = self.convblock6(x)
        x = self.pool2(x)#input - 14; Output - 14; RF - 28; output channels - 25
        x = self.convblock7(x)
        x = self.gap(x)        
        x = self.convblock8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [7]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]              90
              ReLU-2           [-1, 10, 28, 28]               0
       BatchNorm2d-3           [-1, 10, 28, 28]              20
           Dropout-4           [-1, 10, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           1,440
              ReLU-6           [-1, 16, 28, 28]               0
       BatchNorm2d-7           [-1, 16, 28, 28]              32
           Dropout-8           [-1, 16, 28, 28]               0
            Conv2d-9           [-1, 10, 28, 28]             160
        MaxPool2d-10           [-1, 10, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           1,440
             ReLU-12           [-1, 16, 14, 14]               0
      BatchNorm2d-13           [-1, 16, 14, 14]              32
          Dropout-14           [-1,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [8]:
torch.manual_seed(2)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-10.0, 10.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [9]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        # get samples
        data, target = data.to(device), target.to(device)
        # Init
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
        # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.
        
        # Predict
        y_pred = model(data)
        
        # Calculate loss
        loss = F.nll_loss(y_pred, target)
        #train_losses.append(loss)

        # Backpropagation
        loss.backward()
        optimizer.step()
        
        # Update pbar-tqdm
        pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Train_Accuracy={100*correct/processed:0.2f}')
        #train_acc.append(100*correct/processed)

        

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Teast_Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [10]:
from torch.optim.lr_scheduler import StepLR
torch.manual_seed(2)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#scheduler = StepLR(optimizer, step_size=4, gamma=0.1)


EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    #scheduler.step()

  0%|          | 0/1875 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=2.32464337348938 batch_id=1 Train_Accuracy=9.38:   0%|          | 2/1875 [00:00<02:26, 12.79it/s]

EPOCH: 0


loss=0.10650712251663208 batch_id=1874 Train_Accuracy=92.27: 100%|██████████| 1875/1875 [01:56<00:00, 16.12it/s]
loss=0.07032986730337143 batch_id=2 Train_Accuracy=95.83:   0%|          | 2/1875 [00:00<02:02, 15.25it/s]


Test set: Average loss: 0.0829, Teast_Accuracy: 9748/10000 (97.48%)

EPOCH: 1


loss=0.07053113728761673 batch_id=1874 Train_Accuracy=97.35: 100%|██████████| 1875/1875 [01:54<00:00, 16.38it/s]
loss=0.007349410094320774 batch_id=2 Train_Accuracy=98.96:   0%|          | 2/1875 [00:00<01:58, 15.78it/s]

...


Test set: Average loss: 0.0219, Teast_Accuracy: 9922/10000 (99.22%)

EPOCH: 13


loss=0.002876818412914872 batch_id=1874 Train_Accuracy=98.89: 100%|██████████| 1875/1875 [01:53<00:00, 16.49it/s]
loss=0.01352925319224596 batch_id=2 Train_Accuracy=98.96:   0%|          | 2/1875 [00:00<01:58, 15.85it/s] 


Test set: Average loss: 0.0260, Teast_Accuracy: 9911/10000 (99.11%)

EPOCH: 14


loss=0.02468656189739704 batch_id=1874 Train_Accuracy=98.94: 100%|██████████| 1875/1875 [01:55<00:00, 16.26it/s]



Test set: Average loss: 0.0226, Teast_Accuracy: 9923/10000 (99.23%)



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2,2,figsize=(15,10))
axs[0, 0].plot(train_losses)
axs[0, 0].set_title("Training Loss")
axs[1, 0].plot(train_acc[4000:])
axs[1, 0].set_title("Training Accuracy")
axs[0, 1].plot(test_losses)
axs[0, 1].set_title("Test Loss")
axs[1, 1].plot(test_acc)
axs[1, 1].set_title("Test Accuracy")